## Naver 영화리뷰 감정분석 데이터 전처리 

Naver 영화리뷰 감정분석 데이터 전처리 과정 입니다.

이 내용을 확인하기 전 아래 내용을 확인하시기 바랍니다.
- [Sentencepiece를 활용해 Vocab 만들기](https://paul-hyun.github.io/vocab-with-sentencepiece/)

[Colab](https://colab.research.google.com/)에서 실행 했습니다.

#### 0. Pip Install
필요한 패키지를 pip를 이용해서 설치합니다.

In [1]:
!pip install sentencepiece
!pip install wget

     |████████████████████████████████| 1.0MB 33.8MB/s 
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=87929ccf783702fd0ef26b8b70cd09def6d8d24d397c53e729bb2d0f16dd40a7
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


#### 1. Google Drive Mount
Colab에서는 컴퓨터에 자원에 접근이 불가능 하므로 Google Drive에 파일을 올려 놓은 후 Google Drive를 mount 에서 로컬 디스크처럼 사용 합니다.
1. 아래 블럭을 실행하면 나타나는 링크를 클릭하세요.
2. Google 계정을 선택 하시고 허용을 누르면 나타나는 코드를 복사하여 아래 박스에 입력한 후 Enter 키를 입력하면 됩니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# data를 저장할 폴더 입니다. 환경에 맞게 수정 하세요.
data_dir = "/content/drive/My Drive/Data/transformer-evolution"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### 2. Imports

In [0]:
import os
import json
import pandas as pd
import sentencepiece as spm
import wget

#### 3. 폴더의 목록을 확인
Google Drive mount가 잘 되었는지 확인하기 위해 data_dir 목록을 확인 합니다.

In [12]:
for f in os.listdir(data_dir):
  print(f)

kowiki.csv.gz
kowiki.model
kowiki.vocab


#### 4. Vocab 및 입력
[Sentencepiece를 활용해 Vocab 만들기](https://paul-hyun.github.io/vocab-with-sentencepiece/)를 통해 만들어 놓은 vocab을 로딩 합니다.

로딩된 vocab을 이용해 input을 만듭니다.

In [13]:
# vocab loading
vocab_file = f"{data_dir}/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

#### 5. 네이버 영화 분류 데이터 준비

In [14]:
filename = wget.download("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", f"{data_dir}")
print(filename)
filename = wget.download("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", f"{data_dir}")
print(filename)
for f in os.listdir(data_dir):
  print(f)

/content/drive/My Drive/Data/transformer-evolution/ratings_train.txt
/content/drive/My Drive/Data/transformer-evolution/ratings_test.txt
kowiki.csv.gz
kowiki.model
kowiki.vocab
ratings_train.txt
ratings_test.txt


In [0]:
""" train data 준비 """
def prepare_train(vocab, infile, outfile):
    df = pd.read_csv(infile, sep="\t", engine="python")
    with open(outfile, "w") as f:
        for index, row in df.iterrows():
            document = row["document"]
            if type(document) != str:
                continue
            instance = { "id": row["id"], "doc": vocab.encode_as_pieces(document), "label": row["label"] }
            f.write(json.dumps(instance))
            f.write("\n")

In [16]:
prepare_train(vocab, f"{data_dir}/ratings_train.txt", f"{data_dir}/ratings_train.json")
prepare_train(vocab, f"{data_dir}/ratings_test.txt", f"{data_dir}/ratings_test.json")
for f in os.listdir(data_dir):
  print(f)

kowiki.csv.gz
kowiki.model
kowiki.vocab
ratings_train.txt
ratings_test.txt
ratings_train.json
ratings_test.json
